### init the engine

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://mahtin@localhost:5432/mahtin', echo=False)

### declare a schema

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(80))
    fullname = Column(String(80))
    nickname = Column(String(50))
    
    def __repr__(self):
        return f"<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})"

### create the table(s)

In [ ]:
Base.metadata.create_all(engine)

### creating a session class

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine) # the bind can be defined later over Session.configure(bind=engine)
session = Session()

### initiate a session and add objects

In [ ]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='Eddy')
ed_user.fullname
session.add(ed_user) # session is pending
our_user = session.query(User).filter_by(name='ed').all()[0]
our_user
our_user2 = session.query(User).filter_by(name='ed').first()
our_user2

In [ ]:
session.add_all([
...     User(name='wendy', fullname='Wendy Williams', nickname='windy'),
...     User(name='mary', fullname='Mary Contrary', nickname='mary'),
...     User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [ ]:
ed_user.nickname = 'Eddo'

In [ ]:
session.query(User).all()

In [ ]:
session.commit()

In [ ]:
session.query(User.id).all()

## Querying

+ query returns named tuples (after attributes or class in query) for every query object

In [ ]:
for tuple in session.query(User, User.fullname).order_by(User.id):
    print(tuple.User.id, tuple.fullname)

+ label your resulting tuple name using label and aliased

In [ ]:
from sqlalchemy.orm import aliased
alUser = aliased(User, name='alUser')
for tuple in session.query(alUser, User.name.label('the_name'))[1:3]:
    print(tuple.alUser.name, tuple.the_name)

### filter and filter_by
* filter_by gives simple attribute identities
* filter allows complex expressions
* all filter commands are fully chainable

In [ ]:
session.query(User).order_by(User.id).filter_by(fullname='Ed Jones').all()
session.query(User).order_by(User.id).filter_by(fullname='Ed Jones').filter_by(nickname='Eddo').all()

In [ ]:
session.query(User).order_by(User.id).filter(User.fullname == 'Ed Jones').all()
session.query(User).order_by(User.id).filter(User.fullname == 'Ed Jones').filter(User.nickname == 'Eddo').all()

In [ ]:
session.query(User).filter(User.fullname.ilike('%WilL%')).all()

In [ ]:
session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])).all()

In [ ]:
from sqlalchemy import and_, or_
session.query(User).filter(or_(User.name.in_(['ed', 'wendy']), User.name == 'fred', User.nickname == 'windy')).all()

+ you can use full SQL statements with from_statement()

In [ ]:
from sqlalchemy import text
session.query(User).from_statement(text("SELECT * FROM users WHERE name=:value").params(value='ed')).first()

### Relationships

+ ForeignKey sets the connection by restricting own keys to match (and thereby possibly relate to) keys in other tables

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    # here, addresses.user is connected to users.id
    # the table carrying the ForeignKey is the many in the relationship
    user_id = Column(Integer, ForeignKey('users.id'))
    # User.addresses is populated by Address.user where Address.user == User.id
    user = relationship("User", back_populates='addresses')
    
    def __repr__(self):
        return f"<Address(email_address={self.email_address})>"

# this is resolved as a collection (configurable but defaulting to a list)
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")


In [ ]:
Base.metadata.create_all(engine)

In [ ]:
jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')

In [ ]:
jack.addresses = [
...                 Address(email_address='jack@google.com'),
...                 Address(email_address='j25@yahoo.com')]

In [ ]:
jack.addresses
jack.addresses[0]
jack.addresses[0].user
jack.addresses[0].user.name

In [ ]:
session.add(jack)
session.commit()

In [ ]:
jack.addresses

In [ ]:
str(session.query(Address.user))

### Joins
+ joins can be better in cases where filtering looks into two tables as the database does not need to load a full cross join into memory (as below)

In [ ]:
session.query(User.name, Address).\
    filter(User.id==Address.user_id). \
    filter(Address.email_address=='jack@google.com').\
    all()

In [ ]:
for u, a in session.query(User, Address).\
    filter(User.id==Address.user_id).\
    filter(Address.email_address=='jack@google.com').\
    all():
        print(u.name)
        print(a.email_address)

As a default, a `join()` joins two dbs on their Foreign keys

In [ ]:
session.query(User).join(Address).\
    filter(Address.email_address=='jack@google.com').\
    all()

Else, you can use more explicit join statements:

In [ ]:
session.query(User).join(Address, User.id==Address.user_id).all()

In [ ]:
session.query(User).join(User.addresses).all()

session.query(User).join(Address, User.addresses).all()

session.query(User).join('addresses').all()

+ join only retrieves Users, where address field is defined
+ for all Users, you can use outerjoin

In [ ]:
session.query(User).outerjoin(Address).all()

In [ ]:
session.query(User, Address).select_from(Address).join(User).all()

### Aliases
+ for several references to same table you can use aliased()
+ then you join the different aliases of same table to retrieve several values

In [ ]:
from sqlalchemy.orm import aliased
add_alias1 = aliased(Address)
add_alias2 = aliased(Address)
session.query(User.name, add_alias1.email_address, add_alias2.email_address).\
    join(add_alias1, User.addresses).\
    join(add_alias2, User.addresses).\
    filter(add_alias1.email_address == 'jack@google.com').\
    filter(add_alias2.email_address == 'j25@yahoo.com').all()


### subqueries
+ first create a subquery using .subquery() which returns an SQL expression construct
+ here, we use subquery for an aggregate function count()
+ the subquery can be treated like a standard table with attributes accessible as c.

In [ ]:
from sqlalchemy import func
sub_query = session.query(Address.user_id, func.count('*').label('address_count')).\
    group_by(Address.user_id).subquery()
sub_query.c.address_count

In [ ]:
session.query(User, sub_query.c.address_count).\
    outerjoin(sub_query, User.id == sub_query.c.user_id).\
    order_by(User.id).all()

### Exists

In [ ]:
from sqlalchemy.sql import exists
existing = exists().where(Address.user_id != User.id)
session.query(User.name).filter(existing).all()

+ EXISTS can be expresses as any(condition) in SQLalchemy

In [ ]:
session.query(User.name).filter(~User.addresses.any()).all()

In [ ]:
session.query(User.name).filter(User.addresses.any(Address.email_address.like('%google%'))).all()

### Eager Loading
+ if you want to load associated data in a single transaction, you can use selectinload
+ in selectinload, the related data is loaded in a separate select call

In [ ]:
from sqlalchemy.orm import selectinload
session.query(User).options(selectinload(User.addresses)).\
    filter_by(name='jack').first().addresses

+ in joinedload, the data is joined in one select call

In [ ]:
from sqlalchemy.orm import joinedload
session.query(User).options(joinedload(User.addresses)).\
    filter_by(name='jack').first().addresses

+ contains_eager preloads the associated data to be used in chained filters
+ in example, Address.user is eager_loaded to allow filtering with User.name

In [ ]:
from sqlalchemy.orm import contains_eager
session.query(Address).join(Address.user).\
    filter(User.name=='jack').\
    options(contains_eager(Address.user)).all()

### Deleting
+ without cascading, data related to the deleted data will still be available

In [ ]:
session.delete(jack)
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

+ with cascading, linked data is deleted as well
+ here, we reset and declare the User class with cascading

In [ ]:
session.close()
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
     
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    addresses = relationship("Address", back_populates='user',
        cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name, self.fullname, self.nickname)
    
    
class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
Base.metadata.create_all(engine)


+ with cascading activate, all associated data will be removed with the object

In [ ]:
session.delete(jack)
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

### Many to Many Relationship
+ for many2many, you can use an association table to combine both related tables

In [ ]:
from sqlalchemy import Table, Text
# association table
post_keywords = Table('post_keywords', Base.metadata,\
            Column('post_id', ForeignKey('posts.id'), primary_key=True),\
            Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [ ]:
class BlogPost(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)
    
    # here the relationship via the association table is created
    keywords = relationship('Keyword', secondary=post_keywords, back_populates='posts')
    
    # init is optional in SQLalchemy
    def __init__(self, headline, body, author):
        self.author = author
        self.headline = headline
        self.body = body
        
    def __repr__(self):
        return f"BlogPost({self.headline}, {self.body}, {self.author})"
    
class Keyword(Base):
    __tablename__ = 'keywords'
    
    id = Column(Integer, primary_key=True)
    keyword = Column(String(50), nullable=False, unique=True)
    posts = relationship('BlogPost', secondary=post_keywords, back_populates='keywords')
    
    # init is optional in SQLalchemy
    def __init__(self, keyword):
        self.keyword = keyword
    

In [ ]:
BlogPost.author = relationship(User, back_populates='posts')
User.posts = relationship(BlogPost, back_populates="author", lazy="dynamic")

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
wendy = session.query(User).filter_by(name='wendy').first()
post = BlogPost("Wendy's Blog Post", "This is a test", wendy)
session.add(post)

In [ ]:
post.keywords.append(Keyword('wendy'))
post.keywords.append(Keyword('firstpost'))

In [ ]:
session.commit()

In [ ]:
session.query(BlogPost).filter(BlogPost.keywords.any(keyword='firstpost')).all()

In [ ]:
session.query(BlogPost).filter(BlogPost.author==wendy).\
filter(BlogPost.keywords.any(keyword='firstpost')).all()

# Pandas with SQLAlchemy

In [ ]:
data = pd.read_sql_table('users', engine)

In [ ]:
data

In [ ]:
for d in data:
    print(d)